In [2]:
import os

from pyciemss.PetriNetODE.base import BetaNoisePetriNetODESystem
from pyciemss.PetriNetODE.events import Event, StartEvent, LoggingEvent, ObservationEvent, StaticParameterInterventionEvent
import pyciemss

from pyciemss.interfaces import setup_model, reset_model, calibrate, intervene, sample, optimize
from pyciemss.PetriNetODE.interfaces import load_petri_model

In [ ]:
import sys


In [7]:
STARTERKIT_PATH = "../test/models/starter_kit_examples/"
MIRA_PATH = "../test/models/evaluation_examples/"

filename = "CHIME-SIR/model_petri.json"
chime_model_filename = os.path.join(STARTERKIT_PATH, filename)

model = BetaNoisePetriNetODESystem.from_mira(chime_model_filename)

start_event = StartEvent(0.0, {"S": 0.99, "I": 0.01, "R": 0.0})
model.load_event(start_event)

tspan = range(1, 10)
logging_events = [LoggingEvent(t) for t in tspan]
model.load_events(logging_events)

solution = model()

# See that the solution returns a dictionary where each value has length 10, one for each logging event.
assert len(solution["I"]) == len(solution["R"]) == len(solution["S"]) == len(tspan)

solution

{'I': tensor([0.0101, 0.0101, 0.0100, 0.0098, 0.0095, 0.0091, 0.0087, 0.0082, 0.0077]),
 'R': tensor([0.0097, 0.0195, 0.0292, 0.0388, 0.0481, 0.0571, 0.0657, 0.0740, 0.0817]),
 'S': tensor([0.9802, 0.9704, 0.9608, 0.9515, 0.9424, 0.9338, 0.9256, 0.9178, 0.9106])}

In [8]:
# Remove logging events
model.remove_logging_events()

# Add observations
observation_events = [ObservationEvent(1.1, {"S": 0.9, "I": 0.09, "R": 0.01}), 
                      ObservationEvent(2.1, {"S": 0.8, "I": 0.18, "R": 0.02}),
                      ObservationEvent(3.1, {"S": 0.7, "I": 0.27, "R": 0.03}),
                      ObservationEvent(4.1, {"S": 0.6, "I": 0.36, "R": 0.04})]

model.load_events(observation_events)

In [9]:
model._static_events

[StartEvent(time=0.0, initial_state={'S': tensor(0.9900), 'I': tensor(0.0100), 'R': tensor(0.)}),
 ObservationEvent(time=1.100000023841858, observation={'S': tensor(0.9000), 'I': tensor(0.0900), 'R': tensor(0.0100)}),
 ObservationEvent(time=2.0999999046325684, observation={'S': tensor(0.8000), 'I': tensor(0.1800), 'R': tensor(0.0200)}),
 ObservationEvent(time=3.0999999046325684, observation={'S': tensor(0.7000), 'I': tensor(0.2700), 'R': tensor(0.0300)}),
 ObservationEvent(time=4.099999904632568, observation={'S': tensor(0.6000), 'I': tensor(0.3600), 'R': tensor(0.0400)})]

In [10]:
# Show that inference works.

from pyro.infer.autoguide import AutoNormal
from pyro.poutine import block
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro

guide = AutoNormal(model)
optim = Adam({'lr': 0.03})
loss_f = Trace_ELBO(num_particles=1)
verbose = True

svi = SVI(model, guide, optim, loss=loss_f)

pyro.clear_param_store()

for j in range(100):
    # calculate the loss and take a gradient step
    # Passing a data argument to svi.step() will condition the model on the data.
    loss = svi.step()
    if verbose:
        if j % 25 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss))

[iteration 0001] loss: 25.3501
[iteration 0026] loss: 22.4490
[iteration 0051] loss: 21.1329
[iteration 0076] loss: 20.7050


In [11]:
# import AutoGuide
from pyro.infer.autoguide import AutoGuide

AutoGuide.__bases__

(pyro.nn.module.PyroModule,)

In [12]:
# Remove the observation events
model.remove_observation_events()

In [13]:
# Load some static parameter intervention events
model.load_event(StaticParameterInterventionEvent(2.99, "beta", 0.0))
model.load_event(StaticParameterInterventionEvent(4.11, "beta", 10.))

# Load the logging events again
model.load_events(logging_events)

In [14]:
model._static_events

[StartEvent(time=0.0, initial_state={'S': tensor(0.9900), 'I': tensor(0.0100), 'R': tensor(0.)}),
 LoggingEvent(time=1),
 LoggingEvent(time=2),
 StaticParameterInterventionEvent(time=2.990000009536743, parameter=beta, value=0.0),
 LoggingEvent(time=3),
 LoggingEvent(time=4),
 StaticParameterInterventionEvent(time=4.110000133514404, parameter=beta, value=10.0),
 LoggingEvent(time=5),
 LoggingEvent(time=6),
 LoggingEvent(time=7),
 LoggingEvent(time=8),
 LoggingEvent(time=9)]

In [15]:
model._observation_var_names

[]

In [16]:
# use pyro predictive
from pyro.infer import Predictive

# Get the return value of model.
predictions = Predictive(model, guide=guide, num_samples=10)()

In [17]:
predictions

{'a_beta': tensor([1.0687, 1.0809, 1.0395, 1.0667, 1.0757, 0.9896, 1.0584, 1.0661, 1.0438,
         0.9999]),
 'a_gamma': tensor([1.0232, 0.9595, 0.9236, 0.9465, 0.9389, 0.9252, 0.9382, 0.9198, 1.0222,
         0.9282]),
 'I_sol': tensor([[0.0103, 0.0105, 0.0104, 0.0037, 0.6268, 0.2807, 0.1014, 0.0365, 0.0131],
         [0.0111, 0.0122, 0.0130, 0.0050, 0.6416, 0.2910, 0.1119, 0.0429, 0.0164],
         [0.0111, 0.0121, 0.0129, 0.0051, 0.6516, 0.3025, 0.1205, 0.0479, 0.0190],
         [0.0111, 0.0122, 0.0130, 0.0050, 0.6452, 0.2950, 0.1149, 0.0446, 0.0173],
         [0.0113, 0.0126, 0.0136, 0.0053, 0.6462, 0.2957, 0.1160, 0.0454, 0.0178],
         [0.0105, 0.0109, 0.0111, 0.0044, 0.6536, 0.3071, 0.1221, 0.0484, 0.0192],
         [0.0111, 0.0122, 0.0130, 0.0051, 0.6474, 0.2975, 0.1168, 0.0457, 0.0179],
         [0.0114, 0.0128, 0.0140, 0.0056, 0.6506, 0.3006, 0.1202, 0.0479, 0.0191],
         [0.0101, 0.0100, 0.0098, 0.0035, 0.6271, 0.2834, 0.1025, 0.0369, 0.0133],
         [0.0106, 0.011

In [18]:
# SCAFFOLDING FOR DYNAMIC EVENT HANDLING BELOW

import pyro
from torch import Tensor
from torchdiffeq import odeint_event, odeint

Time = Tensor
State = tuple[Tensor, ...]

class BaseODEModel(pyro.nn.PyroModule):

    def __init__(self, static_events: list[Event]):
        super().__init__()
        # This is a list of events that are always sorted by time.
        # TODO: probably pre-sort this list just in case.
        self.static_events = static_events

    def deriv(self, t: Time, state: State) -> State:
        raise NotImplementedError

    @pyro.nn.pyro_method
    def param_prior(self):
        raise NotImplementedError

    @pyro.nn.pyro_method
    def initial_conditions_prior(self):
        raise NotImplementedError

    # @pyro.nn.pyro_method
    # def observation_model(self, state: State, tspan: Time, ?) -> ?: …

    def solve(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        
        current_state = initial_state
        current_time = initial_time

        solution = torch.tensor([])

        for i, static_event in enumerate(self.static_events):
            
            # TODO: change below
            # Note: Immediate goal is to get self.solve() to generate ode solutions that stop at the event times.
            # Note: Next we will add log likelihood at those events using an observation model.

            # Note: each static_event is an Event object, which has a forward() method that returns a Tensor.
            # TODO: Chad, could you please make this actually make sense.
            event_time, event_solution = odeint(self.deriv, current_state, current_time, event_fn=static_event)
    
            # TODO: Add log likelihood for event_solution.
            # Note: This will be done by adding an observation model to the ODE model and calling it with the event_solution as an argument.

            solution = torch.cat([solution, event_solution], dim=-1)
            # current = event_solution[-1]


    def forward(self, initial_state: State, initial_time: Time) -> tuple[State, State]:
        # Sample parameters from the prior. These parameters are generated as attributes of the model.
        self.param_prior()

        # TODO: Sample initial conditions from the prior instead of taking them as input.

        # Solve the ODE, taking into account any intervention and conditioning events.
        return self.solve(initial_state, initial_time)


